In [1]:
from datasets import load_dataset

# Load the MBPP dataset
dataset = load_dataset("mbpp", "full") # or "sanitized" if you prefer

# Initialize max length
max_char_length = 0

# Iterate through the 'test' split (MBPP is often a single split, or you might choose 'train')
# and check the 'code' column.
# Common splits are 'train', 'test', 'validation'. MBPP 'full' has 'train', 'test', 'prompt'.
# We'll check all available splits to be safe if the exact structure isn't recalled.
for split in dataset.keys():
    for example in dataset[split]:
        code_snippet = example['code']
        if code_snippet: # Check if the code snippet is not None or empty
            current_length = len(code_snippet)
            if current_length > max_char_length:
                max_char_length = current_length

print(f"The highest character length in the 'code' column is: {max_char_length}")

/Users/krizroycetahimic/miniconda3/envs/pva_sae/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The highest character length in the 'code' column is: 1331
